In [1]:
import os
import time
import pinecone
import phoenix as px
import pandas as pd

from wrapper.openAiEmbeddingWrapper import OpenAIEmbeddingsWrapper
from wrapper.pineconeWrapper import PineconeWrapper

from utils.evaluation import evaluate_retrievals, process_binary_responses
from utils.customPrompt import get_custom_chain_type_args

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

from utils.prep import generate_products_json
from utils.pinecone import upsert_doc, get_vectorstores_data
from utils.mongodb import get_documents, update_documents
from utils.arizeAI import get_arize_url
from utils.arizeAI import data_prep

/Users/billywitt/.pyenv/versions/3.9.2/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
database_df = get_vectorstores_data()
query_df = pd.DataFrame(list(get_documents()))

print(database_df.columns)
print(query_df.columns)

# px.close_app()

# [query_df, database_df] = data_prep(query_df, database_df)

# query_schema = px.Schema(
#     prompt_column_names=px.EmbeddingColumnNames(
#         raw_data_column_name="text",
#         vector_column_name="centered_text_vector",
#     ),
#     response_column_names="response",
#     tag_column_names=[
#         "context_text_0",
#         "context_similarity_0",
#         "context_text_1",
#         "context_similarity_1",
#         "euclidean_distance_0",
#         "euclidean_distance_1",
#         "openai_relevance_0",
#         "openai_relevance_1",
#         "openai_precision@1",
#         "openai_precision@2",
#     ],
# )

# database_schema = px.Schema(
#     document_column_names=px.EmbeddingColumnNames(
#         raw_data_column_name="text",
#         vector_column_name="centered_text_vector",
#     ),
# )

# # Dataset
# prim_ds = px.Dataset(
#     dataframe=query_df,
#     schema=query_schema,

#     name="query",
# )
# corpus_ds = px.Dataset(
#     dataframe=database_df,
#     schema=database_schema,
#     name="pinecone",
# )

# session = px.launch_app(
#     primary=prim_ds, corpus=corpus_ds, port=6060)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'www-authenticate': 'API key is missing or invalid for the environment "us-west1-gcp". Check that the correct environment is specified.', 'content-length': '114', 'date': 'Wed, 29 Nov 2023 06:57:39 GMT', 'server': 'envoy'})
HTTP response body: API key is missing or invalid for the environment "us-west1-gcp". Check that the correct environment is specified.
